In [66]:
import subprocess
import os
import regex as re
import numpy as np
import itertools
import operator
from collections import Counter
from scipy import stats
from statistics import mean
import gzip
import pickle
import pysam
import mygene
pysam.__version__

'0.15.2'

In [5]:
bamfile = pysam.AlignmentFile("E4_mini_Ig.bam","rb")
no_split = pysam.AlignmentFile("No-split.bam", "wb", template=bamfile)
split = pysam.AlignmentFile("Split.bam", "wb", template=bamfile)
n = 0
for line in bamfile:
    n +=1
    if line.has_tag("SA"):
        split.write(line)
    else:
        no_split.write(line)

no_split.close()
split.close()
bamfile.close()

In [92]:
bamfile = pysam.AlignmentFile("Split.bam")
tabixfile = pysam.TabixFile("coding_gene_region.bed.gz")
out_bam = pysam.AlignmentFile("translocations.bam", "wb", template=bamfile)

for line in bamfile:

    a = line.get_tag("SA").split(";")[0]
    chrom, start, strand, cigar, mapq, nm = a.split(",")
    
    match = re.findall(r'[0-9]+[A-Za-z]', cigar)
    for m in match:
        if "M" in m:
            m = m.replace("M", "")
            if "chrUn" in chrom or "alt" in chrom:
                pass
            else:
                for gtf in tabixfile.fetch(chrom, int(start), int(start)+int(m)):
                    gtf = gtf.split("\t")
                    if gtf[0] == "chrM":
                        pass
                    else:
                        line.tags += [("Ta", gtf[0])]
                        line.tags += [("Tb", gtf[1])]
                        line.tags += [("Tc", gtf[2])]
                        line.tags += [("Td", gtf[3])]

                        out_bam.write(line)
out_bam.close()
bamfile.close()

In [93]:
# now parse translocations and add information regarding the IG

bamfile = pysam.AlignmentFile("translocations.bam","r")
tabixfile = pysam.TabixFile("Ig_regions.bed.gz")
out_bam = pysam.AlignmentFile("translocations_IgAnnot.bam", "wb", template=bamfile)


for line in bamfile:


    start = line.reference_start
    end = line.reference_end
    chrom = line.reference_name

    for gtf in tabixfile.fetch(chrom, int(start), int(end)):
        ig = gtf.split("\t")[3]
        
        line.tags += [("IG", ig)]

        out_bam.write(line)

    
tabixfile.close()
            

    
bamfile.close()
out_bam.close()

In [102]:
# Now need to parse the bam file and generate bed files for each record to make circal plot

bamfile = pysam.AlignmentFile("translocations_IgAnnot.bam","r")
bed1 = open("Ig_features.bed", "w")
bed2 = open("Translocation_features.bed", "w")


for line in bamfile:
    
    barcode = line.query_name.split("_")[1]

    chrom1 = line.reference_name
    start1 = line.reference_start
    end1 = line.reference_end
    ig = line.get_tag("IG")
    

    chrom2 = line.get_tag("Ta")
    start2 = line.get_tag("Tb")
    end2 = line.get_tag("Tc")
    gene = line.get_tag("Td")


    bed1.write("%s\t%s\t%s\t%s\t%s\n" % (chrom1, start1, end1, ig, barcode))
    bed2.write("%s\t%s\t%s\t%s\t%s\n" % (chrom2, start2, end2, gene, barcode))
        
bed1.close()
bed2.close()
bamfile.close()

In [6]:
# %%timeit
import collections

bed1 = open("Ig_features.bed", "r")
bed2 = open("Translocation_features.bed", "r")

out_table = open("annotated_trans_ens.txt", "w")

trans_list = []

#gc.disable()
n = 0
for bed1, bed2 in zip(bed1, bed2):

    barcode = bed1.split("\t")[4]
    ig = bed1.split("\t")[3]
    transloc = bed2.split("\t")[3]
    trans_gene = ig + "_" + transloc + "_"+ barcode.strip()
    trans_list.append(str(trans_gene))
#gc.enable()
out_table.write("Ig\tgene\ttrans\tbarcode\tcount\n")
print(trans_list)
trans_counter =collections.Counter(trans_list)
print(trans_counter)
for k in trans_counter:
    ig, gene, barcode = k.split("_")
    gene2 = ig + "_" + gene
    counter = trans_counter[k]

    out_table.write("%s\t%s\t%s\t%s\t%s\n"%(ig, gene, gene2, barcode, str(counter)))
    
out_table.close()
        

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [37]:
infile = open("annotated_trans_ens.txt", "r")
barcodes = open("jjn3_barcodes.csv", "r")
outf = open("JJN3_counts.txt","w")
barcodes_list = []

for i in barcodes:
    barcodes_list.append(i.strip())
    
n = 0
for i in infile:

    barcode = i.strip().split("\t")[3]
    if barcode in barcodes_list:
        outf.write("%s"%(i))

            
outf.close()
infile.close()
barcodes.close()

In [38]:
infile = open("annotated_trans_ens.txt", "r")
barcodes = open("df15_barcodes.csv", "r")
outf = open("DF15_counts.txt","w")
barcodes_list = []

for i in barcodes:
    barcodes_list.append(i.strip())
    
n = 0
for i in infile:

    barcode = i.strip().split("\t")[3]
    if barcode in barcodes_list:
        outf.write("%s"%(i))
            
            
outf.close()
infile.close()
barcodes.close()

In [28]:
infile = open("annotated_trans_ens.txt", "r")
barcodes = open("h929_barcodes.csv", "r")
outf = open("H929_counts.txt","w")
barcodes_list = []

for i in barcodes:
    barcodes_list.append(i.strip())
    
n = 0
for i in infile:

    barcode = i.strip().split("\t")[3]
    if barcode in barcodes_list:
        outf.write("%s"%(i))
            
            
outf.close()
infile.close()
barcodes.close()

In [62]:
bed1 = open("Ig_features.bed", "r")
bed2 = open("Translocation_features.bed", "r")
barcode = open("jjn3_barcodes.csv", "r")
outf = open("Ig_features_JJN3.bed","w")
outf2 = open("Translocation_features_JJN3.bed","w")

barcode_list = []

for i in barcode:
    barcode_list.append(i.strip())

for bed1, bed2 in zip(bed1, bed2):

    barcode = bed1.split("\t")[4]
    ig = bed1.split("\t")[3]
    barcode = barcode.strip()
    
    if barcode in barcode_list:
        outf.write(bed1)
        outf2.write(bed2)
        
outf.close()
outf2.close()

In [63]:
bed1 = open("Ig_features.bed", "r")
bed2 = open("Translocation_features.bed", "r")
barcode = open("df15_barcodes.csv", "r")
outf = open("Ig_features_DF15.bed","w")
outf2 = open("Translocation_features_DF15.bed","w")

barcode_list = []

for i in barcode:
    barcode_list.append(i.strip())

for bed1, bed2 in zip(bed1, bed2):

    barcode = bed1.split("\t")[4]
    ig = bed1.split("\t")[3]
    barcode = barcode.strip()
    
    if barcode in barcode_list:
        outf.write(bed1)
        outf2.write(bed2)
        
outf.close()
outf2.close()

In [64]:
bed1 = open("Ig_features.bed", "r")
bed2 = open("Translocation_features.bed", "r")
barcode = open("h929_barcodes.csv", "r")
outf = open("Ig_features_H929.bed","w")
outf2 = open("Translocation_features_H929.bed","w")

barcode_list = []

for i in barcode:
    barcode_list.append(i.strip())

for bed1, bed2 in zip(bed1, bed2):

    barcode = bed1.split("\t")[4]
    ig = bed1.split("\t")[3]
    barcode = barcode.strip()
    
    if barcode in barcode_list:
        outf.write(bed1)
        outf2.write(bed2)
        
outf.close()
outf2.close()

In [65]:
bed1 = open("Ig_features.bed", "r")
bed2 = open("Translocation_features.bed", "r")
barcode = open("total_barcodes.csv", "r")
outf = open("Ig_features_total.bed","w")
outf2 = open("Translocation_features_total.bed","w")

barcode_list = []

for i in barcode:
    barcode_list.append(i.strip())

for bed1, bed2 in zip(bed1, bed2):

    barcode = bed1.split("\t")[4]
    ig = bed1.split("\t")[3]
    barcode = barcode.strip()
    
    if barcode in barcode_list:
        outf.write(bed1)
        outf2.write(bed2)
        
outf.close()
outf2.close()